# Semantics

In [1]:
import utils
from minipy import *

Symbolic states are the basic building blocks of symbolic execution. They differ from concrete execution states inasmuch that program variables are assigned symbolic instead of concrete expressions, and that they include an additional path constraint accumulating decisions taken during symbolic execution.  

**Definition** (Symbolic Execution State). A *Symbolic (Execution) State* (SES) is a triple $(\mathit{Constraint}, \mathit{Store}, \mathit{PC})$ of (1) a *path constraint* $\mathit{Constraint}$, formally a set of formulas over program variables, (2) a *symbolic store* $\mathit{Store}$, formally a set of assignments of program variables to symbolic expressions over program variables, and (3) a *program counter* $\mathit{PC}$, formally a minipy program.

Symbolic values in minipy are arithmetic expressions, boolean expressions, or sequences. Instead of defining our own language for symbolic expressions, we go for a shallow embedding into the language of the z3 SMT solver [^z3]. To represent tuples, we use z3's `SeqRef` type for sequences. `SeqRef` does not permit expressing nested sequences. To add nested sequences as symbolic values, we would have to go for a deeper embedding, which we abstain from for simplicity.

In [2]:
from typing import Union
import z3

In [3]:
SymbolicValueType = Union[z3.ArithRef, z3.BoolRef, z3.SeqRef]

We first need a procedure to produce a symbolic z3 expression for a minipy Variable:

In [4]:
class Variable(Variable):
    def to_z3(self):
        if self.type == INT_TYPE:
            return z3.Int(self.name)
        elif self.type == BOOL_TYPE:
            return z3.Bool(self.name)
        elif self.type == TUPLE_TYPE:
            return z3_sequence(self.name)

Consider, for example, the program `x = x + 1`, which increments the initial, symbolic value of the integer variable `x`. The resulting symbolic value is

In [5]:
x = Variable("x", INT_TYPE)
symbolic_increment_expression = x.to_z3() + z3.IntVal(1)
symbolic_increment_expression

x + 1

In [6]:
type(symbolic_increment_expression).__name__

'ArithRef'

A symbolic sequence (which represents a non-nested minipy tuple) is defined the `z3.Unit` and `z3.Concat` primitives. We define a helper function transforming Python integer lists to z3 sequences:

In [7]:
def int_list_to_seq(l: List[int]) -> z3.SeqRef:
    if len(l) == 1:
        return z3.Unit(z3.IntVal(l[0]))
    else:
        return z3.Concat(*[z3.Unit(z3.IntVal(elem)) for elem in l])

In [8]:
int_list_to_seq([1, 2, 3])

Concat(Unit(1), Concat(Unit(2), Unit(3)))

In [9]:
int_list_to_seq([1, 2, 3])[z3.IntVal(1)]

Nth(Concat(Unit(1), Concat(Unit(2), Unit(3))), 1)

In [10]:
z3.simplify(int_list_to_seq([1, 2, 3])[z3.IntVal(1)])

2

We can now implement a symbolic store. In essence, a symbolic store is just a mapping from variables to symbolic values:

In [24]:
class SymbolicStore:
    def __init__(self, env: Optional[Dict[Variable, SymbolicValueType]] = None):
        if env is None:
            self.env: Dict[Variable, SymbolicValueType] = {}
        else:
            self.env = env

    def __eq__(self, other):
        return type(other) is type(self) and self.env == other.env
    
    def __repr__(self):
        return f"SymbolicStore({repr(self.env)})"

We overload the Python's `__getitem__` function to access elements of the store using dictionary syntax `store[variable]`:

In [12]:
class SymbolicStore(SymbolicStore):
    def __getitem__(self, item: Union[str, Variable]):
        if item in self.env:
            return self.env[item]
        else:
            if type(item) is str:
                maybe_var = [variable for variable in self.env if variable.name == item]
                if maybe_var:
                    assert len(maybe_var) == 1
                    return self.env[maybe_var[0]]

            raise AttributeError(f"Attempt to read uninitialized variable {item}")

We also need a setter function. We design symbolic stores to be immutable, which is why the setter returns a new store:

In [13]:
class SymbolicStore(SymbolicStore):
    def set(self, variable: Variable, value: SymbolicValueType) -> 'SymbolicStore':
        new_env = copy.deepcopy(self.env)
        new_env[variable] = value
        return SymbolicStore(new_env)

Back to our `x = x + 1` example: Assume we start symbolic execution in a state where variable `x` is assigned its unknown, initial symbolic value:

In [14]:
store = SymbolicStore({x: x.to_z3()})
store[x]

x

Now, we update the value of `x` to `x + 1`:

In [15]:
store = store.set(x, x.to_z3() + z3.IntVal(1))
store[x]

x + 1

What happens when we do that once again?

In [16]:
store = store.set(x, x.to_z3() + z3.IntVal(1))
store[x]

x + 1

This could have been expected form our definition of the `set` method; however, it is not desirable: Since the current value of `x` was *already* `x + 1`, it should be `x + 2` after adding `1` another time! We have to consider the current assignments of variables occurring in symbolic expressions. In other words, we have to *apply* existing assignments to symbolic expressions. Consequently, we define an `apply_to` method:

In [17]:
def z3_subst(expr: z3.ExprRef, subst_map: Dict[z3.ExprRef, z3.ExprRef]) -> z3.ExprRef:
    return z3.substitute(expr, *tuple(subst_map.items()))


class SymbolicStore(SymbolicStore):
    def apply_to(self, onto: SymbolicValueType) -> SymbolicValueType:
        return z3_subst(onto, {variable.to_z3(): self.env[variable]
                               for variable in self.env})

Let's try it out:

In [18]:
store = SymbolicStore({x: x.to_z3() + z3.IntVal(1)})
store.apply_to(x.to_z3() + z3.IntVal(1))

x + 1 + 1

In [19]:
z3.simplify(store.apply_to(x.to_z3() + z3.IntVal(1)))

2 + x

We use this to correct our `set` method:

In [20]:
class SymbolicStore(SymbolicStore):
    def set(self, variable: Variable, value: SymbolicValueType) -> 'SymbolicStore':
        new_env = copy.deepcopy(self.env)
        new_env[variable] = self.apply_to(value)
        return SymbolicStore(new_env)

In [21]:
store = SymbolicStore({x: x.to_z3() + z3.IntVal(1)})
store[x]

x + 1

In [22]:
store = store.set(x, x.to_z3() + z3.IntVal(1))
z3.simplify(store[x])

2 + x

We call our Python representation of symbolic states *symbolic environments* to align with our concrete minipy interpreter. Analogously to concrete environments, a symbolic environment contains a store and a repository of registered functions. Additionally, it comprises the set of path constraints. We also define getter and setter functions for store elements. The `add_constraint` method for adding a new path constraint applies the current store to that constraint, similarly to the application of the current store to symbolic expressions on updating the value of a variable. Symbolic environments are also immutable.

In [26]:
from typing import Set

In [27]:
class SymbolicEnvironment:
    def __init__(self,
                 store: Optional[SymbolicStore] = None,
                 path_constraints: Optional[Set[z3.BoolRef]] = None,
                 functions: Optional[Dict[str, Tuple[Tuple[Variable], Type, Callable]]] = None):
        self.store: SymbolicStore = SymbolicStore() if store is None else store
        self.path_constraints: Set[z3.BoolRef] = \
            set() if path_constraints is None else simplify_formulas(*path_constraints)
        self.functions: Dict[str, Tuple[Tuple[Variable], Type, Callable]] = \
            {} if functions is None else copy.deepcopy(functions)

    def __getitem__(self, item: Union[str, Variable]) -> SymbolicValueType:
        return self.store[item]

    def set(self, variable: Variable, value: SymbolicValueType) -> 'SymbolicEnvironment':
        return SymbolicEnvironment(self.store.set(variable, value), self.path_constraints,
                                   self.functions)

    def add_constraints(self, *constraints: z3.BoolRef) -> 'SymbolicEnvironment':
        result = self
        for constraint in constraints:
            result = result.add_constraint(constraint)
        return result

    def add_constraint(self, constraint: z3.BoolRef) -> 'SymbolicEnvironment':
        new_constraint = self.store.apply_to(constraint)
        old_constraints = {constraint for constraint in self.path_constraints
                           if not implies(new_constraint, constraint)}
        return SymbolicEnvironment(self.store,
                                   old_constraints | {new_constraint},
                                   self.functions)

    def add_function(self, name: str, params: Tuple[Variable], type: Type, impl: Callable) -> 'SymbolicEnvironment':
        new_functions = copy.deepcopy(self.functions)
        new_functions[name] = (params, type, impl)
        return SymbolicEnvironment(self.store, self.path_constraints, self.abrupt_completion, new_functions)

    def __repr__(self):
        result = f"SymbolicEnvironment({repr(self.store)}, " \
                 f"{repr(self.path_constraints)}, " \
                 f"{repr(self.abrupt_completion)}, "
        result += repr({f_name: f"fun {f_name}{params} -> "
                                f"{type}" for f_name, (params, type, _) in self.functions.items()})
        return f"{result})"

    def __eq__(self, other):
        return type(other) is type(self) and self.store == other.store and \
               self.path_constraints == other.path_constraints and \
               self.functions == other.functions and \
               (self.abrupt_completion == other.abrupt_completion is None or
                ExceptionWrapper(self.abrupt_completion) == ExceptionWrapper(other.abrupt_completion))


## References

```{bibliography}
:filter: docname in docnames
```

[^z3]: https://github.com/Z3Prover/z3